In [2]:
!pip install selenium-stealth


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 임산부

In [ ]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_pregnant_woman.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = (
        "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll"
        "?cityDoArea=ALL&siGunGuArea=ALL&sidocode=ALL&svccd=ALL"
        "&tccd=ALL&meancd=ALL&chktype1=&startCount=0&sortOrder=DESC"
        "&realQuery=%28%EC%9E%84%EC%82%B0%EB%B6%80%29"
        "&sort=RANK&query=%EC%9E%84%EC%82%B0%EB%B6%80&showView=view22"
    )
    driver.get(base_url)

    records = []
    try:
        for page in range(1, 31):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
[페이지 2] 수집 중…
[페이지 3] 수집 중…
[페이지 4] 수집 중…
[페이지 5] 수집 중…
[페이지 6] 수집 중…
[페이지 7] 수집 중…
[페이지 8] 수집 중…
[페이지 9] 수집 중…
[페이지 10] 수집 중…
[페이지 11] 수집 중…
[페이지 12] 수집 중…
[페이지 13] 수집 중…
[페이지 14] 수집 중…
[페이지 15] 수집 중…
[페이지 16] 수집 중…
[페이지 17] 수집 중…
[페이지 18] 수집 중…
[페이지 19] 수집 중…
[페이지 20] 수집 중…
[페이지 21] 수집 중…
[페이지 22] 수집 중…
[페이지 23] 수집 중…
[페이지 24] 수집 중…
[페이지 25] 수집 중…
[페이지 26] 수집 중…
[페이지 27] 수집 중…
[페이지 28] 수집 중…
[페이지 29] 수집 중…
[페이지 30] 수집 중…
완료: 총 704건 → 'gov_services_pregnant_woman.xlsx'


In [19]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_pregnant_woman.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_pregnant_woman.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_pregnant_woman.json"

cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 CSV 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
# CSV 파일이 없으면 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns) + cols).to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1,b=3): time.sleep(random.uniform(a,b))

def safe_click(el):
    try: el.click()
    except: driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수
# ——————————————
def scrape_panel1(idx):
    try:
        el = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner')))
        mapping = {'term':'신청기간','call':'전화문의','reception':'접수기관','support':'지원형태','legalInfo':'사업근거'}
        for cls,col in mapping.items():
            try: df.at[idx,col]=el.find_element(By.CSS_SELECTOR,f'li.{cls} span').text.strip()
            except: df.at[idx,col]=''
    except: pass

# 패널 클릭 후 추출
def scrape_tab(idx,panel):
    safe_click(wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,f"button[data-tab='{panel}']"))))
    human_delay(0.5,1)
    if panel in ['panel2','panel3']:
        sel=f"#{panel} .detail-desc,#{panel} .service-detail-inner"
        try: df.at[idx,{'panel2':'지원대상','panel3':'지원내용'}[panel]] = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, sel))).text.strip()
        except: df.at[idx,{'panel2':'지원대상','panel3':'지원내용'}[panel]]=''
    elif panel=='panel4':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel4 .detail-wrap'):
            t=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '신청기간' in t: df.at[idx,'신청기간']=txt
            if '신청방법' in t: df.at[idx,'신청방법']=txt
    elif panel=='panel5':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel5 .detail-wrap'):
            t=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '접수기관' in t: df.at[idx,'접수기관']=txt
            if '문의처' in t: df.at[idx,'문의처']=txt

# ——————————————
# 4) 메인 루프: 실시간 CSV 기록
# ——————————————
for idx,row in df.iterrows():
    url=row.get('detail_url')
    if not url or checkpoint.get(url,{}).get('finished'): continue
    driver.get(url); human_delay(1,2)
    scrape_panel1(idx)
    for p in ['panel3','panel2','panel4','panel5']: scrape_tab(idx,p)
    missing=[c for c in cols if not df.at[idx,c].strip()]
    out_row = row.to_dict()
    for c in cols: out_row[c]=df.at[idx,c]
    # CSV에 추가
    pd.DataFrame([out_row]).to_csv(OUTPUT_CSV_PATH, mode='a', index=False, header=False, encoding='utf-8-sig')
    if not missing:
        checkpoint[url]={'finished':True}
        with open(CHECKPOINT_PATH,'w',encoding='utf-8') as f: json.dump(checkpoint,f,ensure_ascii=False,indent=2)
        print(f"[{idx+1}/{len(df)}] 저장완료: {url}")
    else:
        print(f"[{idx+1}/{len(df)}] 누락:{missing}")

# ——————————————
# 5) 재시도 및 종료
# ——————————————
# (생략) – 필요 시 위와 동일하게 재시도 구현

driver.quit()
print('모든 작업 완료')

KeyError: '문의처'

### 여성 정책

In [12]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_woman.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll?cityDoArea=ALL&siGunGuArea=&sidocode=&svccd=&tccd=&meancd=&chktype1=&startCount=0&sortOrder=DESC&collection=&range=&startDate=&endDate=&searchField=ALL&reQuery=&stQuery=&downOrgCd=&tmpReQuery=&tmpExReQuery=&realQuery=%28%EC%9E%84%EC%82%B0%EB%B6%80%29%7C%28%EC%9E%84%EC%82%B0%EB%B6%80%29&detailLst=0&sort=RANK&showView=view22&orgSel=ALL&chktype21=&query=%EC%97%AC%EC%84%B1"

    driver.get(base_url)

    records = []
    try:
        for page in range(1, 15):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
[페이지 2] 수집 중…
[페이지 3] 수집 중…
[페이지 4] 수집 중…
[페이지 5] 수집 중…
[페이지 6] 수집 중…
[페이지 7] 수집 중…
[페이지 8] 수집 중…
[페이지 9] 수집 중…
[페이지 10] 수집 중…
[페이지 11] 수집 중…
[페이지 12] 수집 중…
[페이지 13] 수집 중…
[페이지 14] 수집 중…
완료: 총 328건 → 'gov_services_woman.xlsx'


In [ ]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_pregnant_woman.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_pregnant_woman.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_pregnant_woman.json"

cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 CSV 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns) + cols).to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1,b=3): time.sleep(random.uniform(a,b))

def safe_click(el):
    try: el.click()
    except: driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수
# ——————————————
def scrape_panel1(idx):
    """
    panel1에서 주요내용, 신청방법(li.method) 포함 추출
    """
    try:
        el = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner')))
        mapping = {
            'term':'신청기간',
            'call':'전화문의',
            'method':'신청방법',
            'reception':'접수기관',
            'support':'지원형태',
            'legalInfo':'사업근거'
        }
        for cls,col in mapping.items():
            try:
                df.at[idx,col] = el.find_element(By.CSS_SELECTOR,f'li.{cls} span').text.strip()
            except NoSuchElementException:
                # 없으면 빈 문자열 유지
                continue
    except TimeoutException:
        pass

# 각 탭 클릭 후 추출
def scrape_tab(idx,panel):
    safe_click(wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,f"button[data-tab='{panel}']"))))
    human_delay(0.5,1)
    # panel2, panel3: 단일 필드
    if panel in ['panel2','panel3']:
        sel=f"#{panel} .detail-desc,#{panel} .service-detail-inner"
        key={'panel2':'지원대상','panel3':'지원내용'}[panel]
        try:
            df.at[idx,key] = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, sel))).text.strip()
        except TimeoutException:
            pass
    # panel4: 신청방법 상세 (신청기간, 신청방법 보강)
    elif panel=='panel4':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel4 .detail-wrap'):
            title=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '신청기간' in title:
                df.at[idx,'신청기간'] = txt
            elif '신청방법' in title:
                df.at[idx,'신청방법'] = txt
    # panel5: 접수기관, 문의처
    elif panel=='panel5':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel5 .detail-wrap'):
            title=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '접수기관' in title:
                df.at[idx,'접수기관'] = txt
            elif '문의처' in title:
                df.at[idx,'문의처'] = txt

# ——————————————
# 4) 메인 루프: 실시간 CSV 기록
# ——————————————
for idx,row in df.iterrows():
    url=row.get('detail_url')
    if not url or checkpoint.get(url,{}).get('finished'): continue
    driver.get(url); human_delay(1,2)
    # panel1부터 HTML 내 li.method 포함해 신청방법 우선 추출
    scrape_panel1(idx)
    # 순차 탭 클릭하여 보강
    for p in ['panel3','panel2','panel4','panel5']:
        scrape_tab(idx,p)
    # 누락 확인
    missing=[c for c in cols if not df.at[idx,c].strip()]
    # CSV 기록
    out=row.to_dict()
    for c in cols: out[c]=df.at[idx,c]
    pd.DataFrame([out]).to_csv(OUTPUT_CSV_PATH,mode='a',index=False,header=False,encoding='utf-8-sig')
    # 체크포인트 및 로그
    if not missing:
        checkpoint[url]={'finished':True}
        with open(CHECKPOINT_PATH,'w',encoding='utf-8') as f: json.dump(checkpoint,f,ensure_ascii=False,indent=2)
        print(f"[{idx+1}/{len(df)}] 저장완료: {url}")
    else:
        print(f"[{idx+1}/{len(df)}] 누락:{missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')

### 한부모 

In [20]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_single_parent.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll?cityDoArea=ALL&siGunGuArea=&sidocode=&svccd=&tccd=&meancd=&chktype1=&startCount=0&sortOrder=DESC&collection=&range=&startDate=&endDate=&searchField=ALL&reQuery=&stQuery=&downOrgCd=&tmpReQuery=&tmpExReQuery=&realQuery=%281%EC%9D%B8+%EA%B0%80%EA%B5%AC%29%7C%28system%3Anumber%29&detailLst=0&sort=RANK&showView=view22&orgSel=ALL&chktype21=&query=%ED%95%9C%EB%B6%80%EB%AA%A8"

    driver.get(base_url)

    records = []
    try:
        for page in range(1, 63):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


[페이지 1] 수집 중…
[페이지 2] 수집 중…
[페이지 3] 수집 중…
[페이지 4] 수집 중…
[페이지 5] 수집 중…
[페이지 6] 수집 중…
[페이지 7] 수집 중…
[페이지 8] 수집 중…
[페이지 9] 수집 중…
[페이지 10] 수집 중…
[페이지 11] 수집 중…
[페이지 12] 수집 중…
[페이지 13] 수집 중…
[페이지 14] 수집 중…
[페이지 15] 수집 중…
[페이지 16] 수집 중…
[페이지 17] 수집 중…
[페이지 18] 수집 중…
[페이지 19] 수집 중…
[페이지 20] 수집 중…
[페이지 21] 수집 중…
[페이지 22] 수집 중…
[페이지 23] 수집 중…
[페이지 24] 수집 중…
[페이지 25] 수집 중…
[페이지 26] 수집 중…
[페이지 27] 수집 중…
[페이지 28] 수집 중…
[페이지 29] 수집 중…
[페이지 30] 수집 중…
[페이지 31] 수집 중…
[페이지 32] 수집 중…
[페이지 33] 수집 중…
[페이지 34] 수집 중…
[페이지 35] 수집 중…
[페이지 36] 수집 중…
[페이지 37] 수집 중…
[페이지 38] 수집 중…
[페이지 39] 수집 중…
[페이지 40] 수집 중…
[페이지 41] 수집 중…
[페이지 42] 수집 중…
[페이지 43] 수집 중…
[페이지 44] 수집 중…
[페이지 45] 수집 중…
[페이지 46] 수집 중…
[페이지 47] 수집 중…
[페이지 48] 수집 중…
[페이지 49] 수집 중…
[페이지 50] 수집 중…
[페이지 51] 수집 중…
[페이지 52] 수집 중…
[페이지 53] 수집 중…
[페이지 54] 수집 중…
[페이지 55] 수집 중…
[페이지 56] 수집 중…
[페이지 57] 수집 중…
[페이지 58] 수집 중…
[페이지 59] 수집 중…
[페이지 60] 수집 중…
[페이지 61] 수집 중…
[페이지 62] 수집 중…
완료: 총 1476건 → 'gov_services_single_parent.xlsx'


In [ ]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_single_parent.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_single_parent.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_single_parent.json"

cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 CSV 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns) + cols).to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1,b=3): time.sleep(random.uniform(a,b))

def safe_click(el):
    try: el.click()
    except: driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수
# ——————————————
def scrape_panel1(idx):
    """
    panel1에서 주요내용, 신청방법(li.method) 포함 추출
    """
    try:
        el = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner')))
        mapping = {
            'term':'신청기간',
            'call':'전화문의',
            'method':'신청방법',
            'reception':'접수기관',
            'support':'지원형태',
            'legalInfo':'사업근거'
        }
        for cls,col in mapping.items():
            try:
                df.at[idx,col] = el.find_element(By.CSS_SELECTOR,f'li.{cls} span').text.strip()
            except NoSuchElementException:
                # 없으면 빈 문자열 유지
                continue
    except TimeoutException:
        pass

# 각 탭 클릭 후 추출
def scrape_tab(idx,panel):
    safe_click(wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,f"button[data-tab='{panel}']"))))
    human_delay(0.5,1)
    # panel2, panel3: 단일 필드
    if panel in ['panel2','panel3']:
        sel=f"#{panel} .detail-desc,#{panel} .service-detail-inner"
        key={'panel2':'지원대상','panel3':'지원내용'}[panel]
        try:
            df.at[idx,key] = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, sel))).text.strip()
        except TimeoutException:
            pass
    # panel4: 신청방법 상세 (신청기간, 신청방법 보강)
    elif panel=='panel4':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel4 .detail-wrap'):
            title=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '신청기간' in title:
                df.at[idx,'신청기간'] = txt
            elif '신청방법' in title:
                df.at[idx,'신청방법'] = txt
    # panel5: 접수기관, 문의처
    elif panel=='panel5':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel5 .detail-wrap'):
            title=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '접수기관' in title:
                df.at[idx,'접수기관'] = txt
            elif '문의처' in title:
                df.at[idx,'문의처'] = txt

# ——————————————
# 4) 메인 루프: 실시간 CSV 기록
# ——————————————
for idx,row in df.iterrows():
    url=row.get('detail_url')
    if not url or checkpoint.get(url,{}).get('finished'): continue
    driver.get(url); human_delay(1,2)
    # panel1부터 HTML 내 li.method 포함해 신청방법 우선 추출
    scrape_panel1(idx)
    # 순차 탭 클릭하여 보강
    for p in ['panel3','panel2','panel4','panel5']:
        scrape_tab(idx,p)
    # 누락 확인
    missing=[c for c in cols if not df.at[idx,c].strip()]
    # CSV 기록
    out=row.to_dict()
    for c in cols: out[c]=df.at[idx,c]
    pd.DataFrame([out]).to_csv(OUTPUT_CSV_PATH,mode='a',index=False,header=False,encoding='utf-8-sig')
    # 체크포인트 및 로그
    if not missing:
        checkpoint[url]={'finished':True}
        with open(CHECKPOINT_PATH,'w',encoding='utf-8') as f: json.dump(checkpoint,f,ensure_ascii=False,indent=2)
        print(f"[{idx+1}/{len(df)}] 저장완료: {url}")
    else:
        print(f"[{idx+1}/{len(df)}] 누락:{missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')


### 미혼모

In [ ]:
import time
import random
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
from selenium.common.exceptions import TimeoutException

def crawl_gov_services_1_to_100(output_path="gov_services_single_mom.xlsx"):
    # 1) undetected-chromedriver 옵션 세팅
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    ua_list = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(ua_list)}")

    # 2) 브라우저 실행 및 stealth 세팅
    driver = uc.Chrome(options=options)
    stealth(
        driver,
        languages=["ko-KR","ko","en-US","en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    wait = WebDriverWait(driver, 10)

    # 3) 시작 URL (1페이지)
    base_url = "https://www.gov.kr/portal/rcvfvrSvc/svcFind/svcSearchAll?cityDoArea=ALL&siGunGuArea=&sidocode=&svccd=&tccd=&meancd=&chktype1=&startCount=0&sortOrder=DESC&collection=&range=&startDate=&endDate=&searchField=ALL&reQuery=&stQuery=&downOrgCd=&tmpReQuery=&tmpExReQuery=&realQuery=%28%ED%95%9C%EB%B6%80%EB%AA%A8%29%7C%28%29&detailLst=0&sort=RANK&showView=view22&orgSel=ALL&chktype21=&query=%EB%AF%B8%ED%98%BC%EB%AA%A8#none"

    driver.get(base_url)

    records = []
    try:
        for page in range(1, 2):
            # 1페이지가 아니면 해당 페이지 번호 클릭
            if page != 1:
                clicked = False
                while not clicked:
                    try:
                        # 페이지 번호 링크
                        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, str(page))))
                        driver.execute_script("arguments[0].click();", link)
                        clicked = True
                    except TimeoutException:
                        # 페이지 번호가 보이지 않으면 그룹 다음(>) 화살표 클릭
                        nxt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음']")))
                        driver.execute_script("arguments[0].click();", nxt)
                        time.sleep(1)

            # 페이지 로딩 대기
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-item[class*='svc_']")))
            time.sleep(random.uniform(1, 2))
            print(f"[페이지 {page}] 수집 중…")

            # 카드 데이터 추출
            cards = driver.find_elements(By.CSS_SELECTOR, "div.card-item[class*='svc_']")
            for c in cards:
                # region
                re = c.find_elements(By.CSS_SELECTOR, ".card-tag .chip")
                region = re[0].text.strip() if re else ""
                # title & detail_url
                te = c.find_element(By.CSS_SELECTOR, ".card-head .card-title")
                title = te.text.strip()
                detail_url = te.get_attribute("href")
                # period
                pe = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'신청기간')]]/span")
                period = pe[0].text.strip() if pe else ""
                # agency
                ae = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'접수기관')]]/span")
                agency = ae[0].text.strip() if ae else ""
                # contact
                ce = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'전화문의')]]/span")
                contact = ce[0].text.strip() if ce else ""
                # type
                te2 = c.find_elements(By.XPATH, ".//li[strong[contains(text(),'지원형태')]]/span")
                svc_type = te2[0].text.strip() if te2 else ""
                # apply_url
                ae2 = c.find_elements(By.CSS_SELECTOR, ".card-cont a.btn-text")
                apply_url = ae2[0].get_attribute("href") if ae2 else ""

                records.append({
                    "region": region,
                    "title": title,
                    "detail_url": detail_url,
                    "period": period,
                    "agency": agency,
                    "contact": contact,
                    "type": svc_type,
                    "apply_url": apply_url,
                })

    finally:
        driver.quit()

    # 4) DataFrame 변환 & 엑셀 저장
    df = pd.DataFrame(records)
    df.to_excel(output_path, index=False)
    print(f"완료: 총 {len(df)}건 → '{output_path}'")

if __name__ == "__main__":
    crawl_gov_services_1_to_100()


In [ ]:
import os
import time
import json
import pandas as pd
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# ——————————————
# 설정
# ——————————————
EXCEL_INPUT_PATH = r"E:\final_prj\gov_services_single_mom.xlsx"
OUTPUT_CSV_PATH  = r"E:\final_prj\gov_services_single_mom.csv"
CHECKPOINT_PATH  = r"E:\final_prj\gov_services_single_mom.json"

cols = [
    "신청기간", "전화문의", "신청방법",
    "접수기관", "지원형태", "사업근거",
    "지원대상", "지원내용", "문의처"
]

# ——————————————
# 1) 엑셀 읽기 및 CSV 초기화
# ——————————————
df = pd.read_excel(EXCEL_INPUT_PATH)
# CSV 헤더 작성
if not os.path.exists(OUTPUT_CSV_PATH):
    pd.DataFrame(columns=list(df.columns) + cols).to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
# 체크포인트 로드
checkpoint = {}
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, 'r', encoding='utf-8') as f:
        checkpoint = json.load(f)

# ——————————————
# 2) 드라이버 설정
# ——————————————
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
]
options = uc.ChromeOptions()
options.add_argument(f"--user-agent={random.choice(ua_list)}")
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=True)
wait = WebDriverWait(driver, 10)

def human_delay(a=1,b=3): time.sleep(random.uniform(a,b))

def safe_click(el):
    try: el.click()
    except: driver.execute_script("arguments[0].click();", el)

# ——————————————
# 3) 스크래핑 함수
# ——————————————
def scrape_panel1(idx):
    """
    panel1에서 주요내용, 신청방법(li.method) 포함 추출
    """
    try:
        el = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#panel1 .service-detail-inner')))
        mapping = {
            'term':'신청기간',
            'call':'전화문의',
            'method':'신청방법',
            'reception':'접수기관',
            'support':'지원형태',
            'legalInfo':'사업근거'
        }
        for cls,col in mapping.items():
            try:
                df.at[idx,col] = el.find_element(By.CSS_SELECTOR,f'li.{cls} span').text.strip()
            except NoSuchElementException:
                # 없으면 빈 문자열 유지
                continue
    except TimeoutException:
        pass

# 각 탭 클릭 후 추출
def scrape_tab(idx,panel):
    safe_click(wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,f"button[data-tab='{panel}']"))))
    human_delay(0.5,1)
    # panel2, panel3: 단일 필드
    if panel in ['panel2','panel3']:
        sel=f"#{panel} .detail-desc,#{panel} .service-detail-inner"
        key={'panel2':'지원대상','panel3':'지원내용'}[panel]
        try:
            df.at[idx,key] = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, sel))).text.strip()
        except TimeoutException:
            pass
    # panel4: 신청방법 상세 (신청기간, 신청방법 보강)
    elif panel=='panel4':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel4 .detail-wrap'):
            title=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '신청기간' in title:
                df.at[idx,'신청기간'] = txt
            elif '신청방법' in title:
                df.at[idx,'신청방법'] = txt
    # panel5: 접수기관, 문의처
    elif panel=='panel5':
        for w in driver.find_elements(By.CSS_SELECTOR,'#panel5 .detail-wrap'):
            title=w.find_element(By.CSS_SELECTOR,'.detail-title').text
            txt=w.find_element(By.CSS_SELECTOR,'.detail-desc').text.strip()
            if '접수기관' in title:
                df.at[idx,'접수기관'] = txt
            elif '문의처' in title:
                df.at[idx,'문의처'] = txt

# ——————————————
# 4) 메인 루프: 실시간 CSV 기록
# ——————————————
for idx,row in df.iterrows():
    url=row.get('detail_url')
    if not url or checkpoint.get(url,{}).get('finished'): continue
    driver.get(url); human_delay(1,2)
    # panel1부터 HTML 내 li.method 포함해 신청방법 우선 추출
    scrape_panel1(idx)
    # 순차 탭 클릭하여 보강
    for p in ['panel3','panel2','panel4','panel5']:
        scrape_tab(idx,p)
    # 누락 확인
    missing=[c for c in cols if not df.at[idx,c].strip()]
    # CSV 기록
    out=row.to_dict()
    for c in cols: out[c]=df.at[idx,c]
    pd.DataFrame([out]).to_csv(OUTPUT_CSV_PATH,mode='a',index=False,header=False,encoding='utf-8-sig')
    # 체크포인트 및 로그
    if not missing:
        checkpoint[url]={'finished':True}
        with open(CHECKPOINT_PATH,'w',encoding='utf-8') as f: json.dump(checkpoint,f,ensure_ascii=False,indent=2)
        print(f"[{idx+1}/{len(df)}] 저장완료: {url}")
    else:
        print(f"[{idx+1}/{len(df)}] 누락:{missing}")

# ——————————————
# 5) 종료
# ——————————————
driver.quit()
print('모든 작업 완료')